In [1]:
import os
import pandas as pd

## Get block
The default training logs containg a block of text at the end that includes all metrics for all experiments and folds. This function gets and returns this block

In [2]:
def get_metric_block(text):
    blocks = []

    block = []
    c = 0
    for line in text.split("\n"):
        if len(line) == 0:
            if len(block) > 0:
                blocks.append(block)
            block = []
        else:
            block.append(line)
            
    return blocks[-1]

## Get metrics
Given the text block obtained above, this function reads the text and parse it to float arrays for each metric and scenario

In [3]:
def get_metrics(metric_block):
    data = []

    for metric in metric_block:
        metric_label,metric_data = metric.split(":")
        
        metric_data = metric_data.split("\t")
        metric_data = [float(i) for i in metric_data]

        if metric_label == "training-time":
            dataset_part = "validation"
            metric_name = "time"
            type_data = "time"
        else:
            dataset_part,metric_name,type_data = metric_label.split("-")
       
        i = 0
        for metri_data_i in metric_data:
            metric_input = [dataset_part,metric_name,type_data,metri_data_i,i]

            data.append(metric_input)
            
            i +=1

    return data

## Creating a csv
The following blocks reads all files from a folder and adds it to a pandas Dataframe including model, dataset, metric, missing pattern (Mode), and fold
then saves the dataframe in a csv file

In [4]:
dataset = []
for path in os.listdir("trained_models"):
    if path.split(".")[-1] != "log":
        continue
    if "svm" in path: continue # Here it is ignoring logs from svm model
    with open("trained_models/{}".format(path)) as file:
        text = file.read()
    
    path = path.split(".log")[0]
    model_name,dataset_name = path.split("_")
    
    try:
        metric_block = get_metric_block(text)
        data_i = get_metrics(metric_block)
    except:
        print(path)
        continue
        
    data_i = [[model_name,dataset_name] + i for i in data_i]
    
    dataset = dataset + data_i

In [5]:
dataset = pd.DataFrame(dataset,columns = ["Model","Dataset","Division type","Metric","Mode","Value","Fold"])
dataset

,Model,Dataset,Division type,Metric,Mode,Value,Fold
0,saits,teapot,validation,time,time,734.0000,0
1,saits,teapot,validation,time,time,859.0000,1
2,saits,teapot,validation,time,time,643.0000,2
3,saits,teapot,validation,time,time,681.0000,3
4,saits,teapot,validation,time,time,659.0000,4
...,...,...,...,...,...,...,...
2830,rf,taranaki,validation,cc,profile,0.3634,0
2831,rf,taranaki,validation,cc,profile,0.3164,1
2832,rf,taranaki,validation,cc,profile,0.3345,2
2833,rf,taranaki,validation,cc,profile,0.3315,3


In [6]:
dataset.to_csv("metrics.csv",index=False)